In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()

from geopy import distance


## Data from zenodo archive

https://zenodo.org/record/7923702

In [2]:
import traffic

from traffic.data import opensky
print(traffic.config_file)

C:\Users\a.salgas.ISAE-SUPAERO\AppData\Local\miniconda3\envs\aeroscope_OS\lib\site-packages\pyopensky\impala.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


C:\Users\a.salgas.ISAE-SUPAERO\AppData\Local\traffic\traffic\traffic.conf


In [3]:
f = open(traffic.config_file, 'r')
file_contents = f.read()
print(file_contents)

## This file contains configuration information for the Python traffic library
##   Documentation page: https://traffic-viz.github.io/
##   Github repository: https://github.com/xoolive/traffic

## Lines prefixed by a # are commented.
## Lines prefixed by a double ## are documentation.

[global]

## aixm_path and nm_path refer to two sources of aeronautical information
## provided by Eurocontrol. Data is updated every AIRAC cycle.
## https://traffic-viz.github.io/airac_usage.html

aixm_path =
nm_path =

[opensky]

## Input here your credentials for accessing the OpenSky Impala shell
## https://opensky-network.org/impala-guide

username = antoineSGS
password = VXa9!8n4YV

## You may set a ssh ProxyCommand in the [network] section

[aircraft]

## By default, aircraft are downloaded from the OpenSky aircraft database
## https://opensky-network.org/aircraft-database

## Specify here the path to your own aircraft database file, in a format that
## pandas can read (.csv, .pickle, etc.)
## Re

In [4]:
flight = opensky.history(
    "2017-02-05 15:45",
    stop="2017-02-05 16:45",
    callsign="EZY158T",
    # returns a Flight instead of a Traffic
    return_flight=True
)


TimeoutError: [WinError 10060] Une tentative de connexion a échoué car le parti connecté n’a pas répondu convenablement au-delà d’une certaine durée ou une connexion établie a échoué car l’hôte de connexion n’a pas répondu

In [ ]:
flights

In [25]:
opensky_2019=opensky_extract.copy()

In [26]:
keep_time=False


# selecting only the columns interesting us
opensky_2019 = opensky_2019.loc[:, ['day', 'destination', 'origin', 'icao24']]
# Drop flight with no known origin and destination (mandatory because of our computation method)
size_before_drops = len(opensky_2019.index)
# opensky_2019.dropna(subset=["destination", "origin"], inplace=True)
size_after_drops = len(opensky_2019.index)
print('{}% of flights deleted after removing '
    'flights with no origin and destination. {} Flights in the dataset'.format(
        (size_before_drops - size_after_drops) / size_before_drops * 100, size_after_drops))

# Opensky only gets aircraft transponder code, not its registration.
# Hopefully, it can be merged with an aircraft database, for example opensky aircraft database

ac_ref=pd.read_csv('../04_bis_opensky_standalone/data/aircraft-database-complete-2022-05.csv', sep=';')


 # Even if each aircraft has a unique icao24 code, some are present several times in the database.
# Duplicates are therefore dropped before merging the dataframes.
ac_ref = ac_ref[['icao24', 'typecode', 'operator_os', 'operatoricao']].drop_duplicates(subset='icao24',
                                                                                       keep='last')
opensky_2019 = pd.merge(opensky_2019, ac_ref, left_on='icao24', right_on='icao24', how='left', sort=False)

# we use ailine ICAO code, but it is named IATA in the following for compatibility reasons
opensky_2019.rename(columns={"operatoricao": "airline_iata",
                        "typecode": "aircraft_type"}, inplace=True)
# opensky_2019.aircraft_type = opensky_2019.aircraft_type.replace("ZZZZ", "zzz")
opensky_2019.aircraft_type.fillna('Unkwown', inplace=True)
opensky_2019.airline_iata.fillna('Unkwown', inplace=True)

# # Counting flight per aircraft type /route/airline each month
if keep_time:
    # Converting dates to a suitable format
    opensky_2019['year'] = opensky_2019.apply(lambda x: get_month_year(x['day'])[0], axis=1)
    opensky_2019['month'] = opensky_2019.apply(lambda x: get_month_year(x['day'])[1], axis=1)

    opensky_2019 = opensky_2019.groupby(
        ['airline_iata', 'origin', 'destination', 'aircraft_type', 'month', 'year'],
        as_index=False, dropna=False).size()
else:
    # allows to significantly reduce the size of the data by eliminating the temporal variables
    #         by aggregating whole dataframe values
    opensky_2019 = opensky_2019.groupby(
        ['airline_iata', 'origin', 'destination', 'aircraft_type'],
        as_index=False, dropna=False).size()

opensky_2019.rename(columns={"size": "n_flights"}, inplace=True)
print(
    'Size of df after_grouping: {}; number of flights: {}'.format(len(opensky_2019.index), opensky_2019.n_flights.sum()))


ac_cla = pd.read_csv('../03_routes_schedule/data/open_sky/aircraft_classification.csv', sep=';')
opensky_2019 = opensky_2019.merge(ac_cla, left_on='aircraft_type', right_on='aircraft_osky', how='left')
opensky_2019.drop(columns=['aircraft_osky'], inplace=True)

opensky_2019.acft_icao.fillna('Unknown', inplace=True)
opensky_2019.acft_class.fillna('Unknown', inplace=True)
opensky_2019.seymour_proxy.fillna('zzz', inplace=True)

print('Size of df after ac info: {}; number of flights {}'.format(len(opensky_2019.index), opensky_2019.n_flights.sum()))

# similarly, we merge an airport database to have info on airport iata designator, and gps coordinates.
# All major airports have IATA designator.

arpt_ref = pd.read_csv('../03_routes_schedule/data/ourairports.csv', sep=';', keep_default_na=False, na_values='')

opensky_2019 = opensky_2019.merge(arpt_ref[['ident', 'iata_code', 'longitude_deg',
                                  'latitude_deg', 'iso_country', 'continent']], left_on='origin',
                        right_on='ident', how='left', sort=False)
opensky_2019.rename(columns={'longitude_deg': 'origin_lon', 'latitude_deg': 'origin_lat',
                        'iso_country': 'origin_country', 'continent': "origin_continent",
                        "iata_code": "origin_iata"}, inplace=True)

opensky_2019[['origin_lon', 'origin_lat',
                        'origin_country', "origin_continent",
                        "origin_iata"]]=opensky_2019[['origin_lon', 'origin_lat',
                        'origin_country', "origin_continent",
                        "origin_iata"]].fillna('Unknown Origin')

opensky_2019 = opensky_2019.merge(arpt_ref[['ident', 'iata_code', 'longitude_deg',
                                  'latitude_deg', 'iso_country', 'continent']], left_on='destination',
                        right_on='ident', how='left', sort=False)
opensky_2019.rename(columns={'destination':'dest','longitude_deg': 'dest_lon', 'latitude_deg': 'dest_lat',
                        'iso_country': 'dest_country', 'continent': "dest_continent", "iata_code": "dest_iata"},
               inplace=True)

opensky_2019[['dest_lon', 'dest_lat',
                        'dest_country', "dest_continent",
                        "dest_iata"]]=opensky_2019[['dest_lon', 'dest_lat',
                        'dest_country', "dest_continent",
                        "dest_iata"]].fillna('Unknown Destination')

opensky_2019.drop(
    columns=['ident_x', 'ident_y'], inplace=True)


opensky_2019.dropna(subset=["dest_lon", "origin_lon", "dest_lat", "origin_lat"], inplace=True)

print(
    'Size of df after arpt info: {}, number of flights: {}'.format(len(opensky_2019.index), opensky_2019.n_flights.sum()))


0.0% of flights deleted after removing flights with no origin and destination. 31218164 Flights in the dataset


C:\Users\a.salgas.ISAE-SUPAERO\AppData\Local\Temp\ipykernel_20284\2013967345.py:17: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  ac_ref=pd.read_csv('../04_bis_opensky_standalone/data/aircraft-database-complete-2022-05.csv', sep=';')


Size of df after_grouping: 2150912; number of flights: 31218164
Size of df after ac info: 2150912; number of flights 31218164
Size of df after arpt info: 2150912, number of flights: 31218164


In [27]:
os_df=opensky_2019.copy()

os_df

,airline_iata,origin,dest,aircraft_type,n_flights,acft_icao,acft_class,seymour_proxy,origin_iata,origin_lon,origin_lat,origin_country,origin_continent,dest_iata,dest_lon,dest_lat,dest_country,dest_continent
0,AAB,BIRK,NaN,CL35,1,CL35,PJ,zzz,RKV,-21.940599,64.129997,IS,EU,Unknown Destination,Unknown Destination,Unknown Destination,Unknown Destination,Unknown Destination
1,AAB,EBAW,EBMB,CL35,1,CL35,PJ,zzz,ANR,4.46028,51.1894,BE,EU,Unknown Destination,4.49,50.913,BE,EU
2,AAB,EBAW,EDDN,CL35,1,CL35,PJ,zzz,ANR,4.46028,51.1894,BE,EU,NUE,11.078056,49.498699,DE,EU
3,AAB,EBAW,EGLF,CL35,1,CL35,PJ,zzz,ANR,4.46028,51.1894,BE,EU,FAB,-0.776333,51.275799,GB,EU
4,AAB,EBAW,NaN,CL35,1,CL35,PJ,zzz,ANR,4.46028,51.1894,BE,EU,Unknown Destination,Unknown Destination,Unknown Destination,Unknown Destination,Unknown Destination
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2150907,bfx,NaN,LIMN,PRM1,1,PRM1,PJ,zzz,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Destination,8.66922,45.529598,IT,EU
2150908,bfx,NaN,LOWW,PRM1,1,PRM1,PJ,zzz,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Origin,VIE,16.5697,48.110298,AT,EU
2150909,bfx,NaN,LSZH,PRM1,1,PRM1,PJ,zzz,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Origin,ZRH,8.548056,47.458056,CH,EU
2150910,bfx,NaN,UKKK,PRM1,1,PRM1,PJ,zzz,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Origin,Unknown Origin,IEV,30.45194,50.40194,UA,EU


In [28]:
fleet=pd.read_excel('data/planespotters_fleet.xlsx')
fleet['Seat Total']=fleet['Seat Total'].replace(0, np.nan)
fleet.dropna(subset=['Seat Total'], inplace=True)

# Adding the name/icao code match table
match=pd.read_csv('data/fleet_match.csv', sep=';')
fleet=fleet.merge(match, left_on='Aircraft Type', right_on='spotter_name', how='right').drop(columns='spotter_name')

aircraft_data=fleet.groupby(['Aircraft Type','ICAO_AC'])['Seat Total'].mean().reset_index().drop(columns='Aircraft Type')

In [29]:
aircraft_data.head()

,ICAO_AC,Seat Total
0,AT43,46.897119
1,AT45,46.897119
2,AT72,70.137580
3,AT75,70.137580
4,AT76,70.137580


In [30]:
os_df=os_df.merge(aircraft_data, left_on='acft_icao', right_on='ICAO_AC', how='left')
os_df['Seat Total']=os_df['n_flights']*os_df['Seat Total']
os_df=os_df.groupby(['origin','dest','origin_iata','origin_lon','origin_lat','origin_country','origin_continent','dest_iata','dest_lon','dest_lat','dest_country','dest_continent','airline_iata','acft_icao','acft_class','seymour_proxy'])[['Seat Total','n_flights']].sum().reset_index().rename(columns={'Seat Total':'seats'})

In [31]:
column_mapping = {
    'origin_iata': 'iata_departure',
    'dest_iata': 'iata_arrival',
    'origin_lon': 'departure_lon',
    'origin_lat': 'departure_lat',
    'origin_country': 'departure_country',
    'origin_continent': 'departure_continent',
    'dest_lon': 'arrival_lon',
    'dest_lat': 'arrival_lat',
    'dest_country': 'arrival_country',
    'dest_continent': 'arrival_continent'
}                          


os_df.rename(columns=column_mapping, inplace=True)
os_df['source']='OpenSky'

In [32]:
iata_icao_convert=pd.read_csv('data/iata_icao_airline.csv', sep=';')
iata_icao_convert=iata_icao_convert.sort_values(by=['IATA'], ascending=False).drop_duplicates(subset='ICAO', keep='first')

In [33]:
os_df=os_df.merge(iata_icao_convert[['IATA','ICAO']], left_on='airline_iata', right_on='ICAO', how='left')

In [34]:
os_df['IATA']=os_df['IATA'].fillna(os_df['airline_iata'])
os_df=os_df.drop(columns=['airline_iata','ICAO']).rename(columns={'IATA':'airline_iata'})

In [ ]:
os_df.loc[:, "distance_km"] = os_df.progress_apply(
            lambda x: distance.distance((float(x.departure_lat), float(x.departure_lon)),
                                        (float(x.arrival_lat), float(x.arrival_lon))).km if not (pd.isna(x.departure_lat) or pd.isna(x.departure_lon) or 
                                                                                                 (x.departure_lat=='Unknown Origin') or (x.departure_lon=='Unknown Origin')
                                                                                           or pd.isna(x.arrival_lon) or pd.isna(x.arrival_lat) or
                                                                                                (x.arrival_lat=='Unknown Destination') or (x.arrival_lon=='Unknown Destination')
                                                                                                ) else 0, axis=1)


  0%|          | 0/1796462 [00:00<?, ?it/s]

In [ ]:
os_df
os_df['ask'] = os_df["distance_km"]*os_df["seats"]
os_df['rpk'] = os_df['ask']*0.824

Load factor source: https://www.icao.int/annual-report-2019/Pages/the-world-of-air-transport-in-2019.aspx

In [ ]:
os_df['seats'].sum()

## Fuel burn computation process

Problem: we do not have a seymour proxi for all the aircraft types neither do we have the seats associated to each aircrfat like in AeroSCOPE core usage. 
One could extend the proxi list, but it is not without increasing the risk of errors.  
Helicopters are particularly often in this case as shown below, followed py piston private and private jets and few turbo props.  
Fortunately, this is not the majority of the flights.

If the interested reader is willing to increase the proxy list for a particular use case, he is free to do so ;) 

In [ ]:
os_df[os_df.seymour_proxy=='zzz'].groupby('acft_class')['n_flights'].sum().nlargest(50)

In [ ]:
os_df[os_df.seymour_proxy!='zzz'].groupby('acft_class')['n_flights'].sum().nlargest(50)

Now, let's compute the fuel burn using the surrogate.

In [ ]:
fuel_surrogate = pd.read_csv('data/FuelSurrogate.csv', sep=';')
fuel_surrogate_dict=fuel_surrogate[['ac_code_icao',"reduced_fuel_intercept","reduced_fuel_a1","reduced_fuel_a2"]].set_index('ac_code_icao').transpose().to_dict()
ac_list=fuel_surrogate['ac_code_icao'].unique()


def compute_fuel_seymour(fuel_surrogate_dict, icao_type, distance):
    if icao_type in ac_list and distance > 1:
        coefficients = fuel_surrogate_dict[icao_type]
        fuel_burn = coefficients['reduced_fuel_intercept'] + coefficients['reduced_fuel_a2'] * distance + coefficients['reduced_fuel_a1'] * distance ** 2
    elif distance < 1:
        fuel_burn = 0
    else:
        fuel_burn = np.nan
    return float(fuel_burn)


os_df.loc[:, "fuel_burn_seymour"] = os_df.progress_apply(
                lambda x: compute_fuel_seymour(fuel_surrogate_dict, x["seymour_proxy"], x["distance_km"]),
                axis=1)

In [ ]:
os_df

In [ ]:
os_df['fuel_burn']=os_df['fuel_burn_seymour']*os_df['n_flights']
os_df['co2']=os_df['fuel_burn']*3.16

os_df['domestic']=(os_df.departure_country == os_df.arrival_country).astype(int)

In [ ]:
os_df.to_csv('data/os_alone_test.csv')